In [1]:
import pandas as pd
import sys
sys.path.append('../gtm/')
from corpus import GTMCorpus
from gtm import GTM
import pickle as p

import os
language = 'en-zh'

/Users/guanlan/miniconda3/envs/gtm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_examples(language='en'):
	df = pd.read_csv('../data/wiki_shorts/{}/corpus/docs.txt'.format(language), header=None, delimiter='\t')
	df.columns = ['doc_clean']
	# df = df.head(1000)
	return df

In [5]:
if not os.path.exists('train_dataset_intfloat-e5-large2-{}.pkl'.format(language)):
# if True:
	df_en = load_examples('en')
	df_zh = load_examples('zh')

	# merge two dfs into one df, with additional column specifying the language
	df_en['language'] = 'en'
	df_zh['language'] = 'zh'

	# randomly select 100 documents from each language
	# df_en = df_en.sample(n=100)
	# df_zh = df_zh.sample(n=100)

	# Concatenate the two DataFrames
	df = pd.concat([df_en, df_zh], ignore_index=True)

	train_dataset = GTMCorpus(
		df,
		count_words=True,
		embeddings_type='SentenceTransformer',
		sbert_model_to_load='intfloat/multilingual-e5-large',
		content=None,
		batch_size=64,
		max_seq_length=512)
	print('Saving train_dataset_intfloat-e5-large2-{}.pkl'.format(language))
	with open('train_dataset_intfloat-e5-large2-{}.pkl'.format(language), 'wb') as f:
		p.dump(train_dataset, f)
else:
	print('Loading train_dataset_intfloat-e5-large2-{}.pkl'.format(language))
	with open('train_dataset_intfloat-e5-large2-{}.pkl'.format(language), 'rb') as f:
		train_dataset = p.load(f)

print('dataset loaded')

Loading train_dataset_intfloat-e5-large2-en-zh.pkl
dataset loaded


In [4]:
# Train the model
tm = GTM(
	train_dataset,
	n_topics=6,
	doc_topic_prior='dirichlet', # logistic_normal, dirichlet
	alpha=0.02,
	update_prior=False,
	encoder_input='embeddings', # 'bow', 'embeddings'
	encoder_hidden_layers=[], # structure of the encoder neural net
	decoder_hidden_layers=[256], # structure of the decoder neural net
	encoder_bias=True,
	decoder_bias=True,
	num_epochs=100,
	print_every=10000,
	dropout=0.0,
	learning_rate=0.01,
	log_every=1,
	w_prior=None,
	batch_size=256,
	patience=20
)


Epoch   1	Mean Training Loss:14591.9225339
Topic_0: ['台湾', '中国', '美国', '音乐', '他们', '大学', '年月', '国家', '主要', '电影', '自己', '香港', '因为', '开始', '成为', '日本', '学校', '主义', '第一', '认为', '乐团', '中华', '国际', '英语', '学名', '制作', '运动', '其他', '政府', '上映', '获得', '专辑', '作品', '担任', '地区', '组织', '政治', '可以', '之后', '教会', '使用', '进行', '同时', '公司', '其中', '发现', '世界', '因此', '历史', '之一', '社会', '以及', '分布', '所有', '不同', '包括', '一般', '称为', '没有', '动画', '最后', '世纪', '独立', '由于', '品种', '创作', '民国', '新闻', '穆斯林', '作为', '方式', '成立', '人民', '演出', '故事', '伊斯兰教', '它们', '父亲', '台北', '工作', '动物', '发展', '宗教', '生活', '期间', '牠们', '民族', '小说', '加入', '导演', '剧情', '出现', '第二', '这个', '节目', '已经', '物种', '后来', '法律', '当时', '中央', '参与', '英国', '伊斯兰', '共和国', '系列', '教宗', '食用', '进入', '通过', '代表', '教育', '植物', '许多', '时间', '大陆', '目前', '然后', '总统', '朝鲜', '联盟', '配音', '基督教', '关系', '歌手', '一些', '举行', '具有', '亚洲', '革命', '正式', 'the', '发行', '2013年', '亚种', '通常', '印度', '生产', '帝国', '为了', '名字', '内容', '但是', '英文', '环境', '来到', '德国', '并且', '特征', '表演', '用来', '成员', '阿拉伯语', '一起', '传统', '会议'

In [5]:
import numpy as np

def inspect(tm, ds):
	doc_topic_distribution = tm.get_doc_topic_distribution(ds)

	print('Number of documents per topic')
	print('Topic 0: {}'.format((doc_topic_distribution.argmax(-1) == 0).sum()))
	print('Topic 1: {}'.format((doc_topic_distribution.argmax(-1) == 1).sum()))
	print('Topic 2: {}'.format((doc_topic_distribution.argmax(-1) == 2).sum()))
	print('Topic 3: {}'.format((doc_topic_distribution.argmax(-1) == 3).sum()))
	print('Topic 4: {}'.format((doc_topic_distribution.argmax(-1) == 4).sum()))
	print('Topic 5: {}'.format((doc_topic_distribution.argmax(-1) == 5).sum()))

	# show five random documents per topic
	for topic in range(tm.n_topics):
		print('Topic {}'.format(topic))
		print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
		for i in np.random.choice(np.where(doc_topic_distribution.argmax(-1) == topic)[0], 5):
			print('=' * 50)
			print(ds.df.iloc[i]['doc_clean'])
			print('----------')
			print('Topic distribution = {}'.format(doc_topic_distribution[i]))

In [6]:
'''
Epoch  53	Mean Training Loss:1832.5807679
'''

'\nEpoch  53\tMean Training Loss:1832.5807679\n'

In [7]:
tm

In [8]:
inspect(tm, train_dataset)

Number of documents per topic
Topic 0: 3619
Topic 1: 3384
Topic 2: 3540
Topic 3: 3338
Topic 4: 3673
Topic 5: 3624
Topic 0
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
高沟捆蹄 又 称 涟水 捆蹄 是 一 种 以 猪肉 为 主要 原料 制作 而 成 是 食物 。 高沟捆蹄 起源于 江苏 涟水 高 沟 地区 是 苏菜徐 海菜 的 特色 小食 之一 。 高 沟 捆蹄 是 一 道 汉族 名菜 属于 苏菜系 此 菜 色泽 酱 红 咸 鲜 香甜 鲜嫩 可口 食 之 不 腻 细嚼 慢 品 回味无穷 。 以 猪肉 为 制作 主料 高沟捆蹄 的 烹饪 技巧 以 卤菜 为主 口味 属于 咸 甜味 。 此 菜 为 涟水县 高沟镇 特产 。 自古 就 流传 着 中秋 向后 日渐 凉高沟捆蹄 相争 尝 购货 客人 如云 涌 只 悉 生产 供 不 上 的 民谣 。 历史 高沟捆蹄 距今 已 有 多 年 的 历史 。 相传 在 清朝 道光 年间 涟水 高 沟 地区 为 商贸 重镇 。 这里 市场 活跃 南来 北往 商贾 众多 。 高 沟 熏 烧 师傅 郑云福 兄弟 为了 给 客人 多 添 一 样 下 酒菜 便 精心 研制 了 高沟捆蹄 因 其 色香味美 一时间 名噪 南北 。
----------
Topic distribution = [9.99913812e-01 1.22524057e-07 1.00892386e-07 7.73376996e-06
 4.92835079e-06 7.32833723e-05]
印度 孔雀龟 （ Morenia petersi ） ， 又 名 印度 沼龟 ， 是 一 个 在 印度 东北部 及 孟加拉 被 发现 的 龟种 （ 在 印度 西部 比哈尔邦 亦 有分佈 ） 。 其 名字 是 由 德国 爬虫学家 威廉·彼得斯 （ 1815-1883 ） 的 名字 而 命名 。 形态 印度 孔雀龟 是 缅甸 孔雀龟 的 近亲 ， 但 口鼻部 较 后者 为 尖 ， 相对 有 较长 的 吻部 。 甲壳 是 绿 、 黑色 的 ， 每 块 甲壳 都 有 一 条 狭窄 

In [9]:
# write a function to write all examples of the same topic to a file
def write_topic_to_file(topic_id, ds, path):
	doc_topic_distribution = tm.get_doc_topic_distribution(ds)
	with open(path, 'w') as f:
		for i in np.where(doc_topic_distribution.argmax(-1) == topic_id)[0]:
			f.write(ds.df.iloc[i]['doc_clean'] + '\n')

In [10]:
write_topic_to_file(0, train_dataset, 'topic_0.txt')
write_topic_to_file(1, train_dataset, 'topic_1.txt')
write_topic_to_file(2, train_dataset, 'topic_2.txt')
write_topic_to_file(3, train_dataset, 'topic_3.txt')
write_topic_to_file(4, train_dataset, 'topic_4.txt')
write_topic_to_file(5, train_dataset, 'topic_5.txt')